In [1]:
from collections import namedtuple 
import shutil, time
from src.config.spark_manager import spark_session
from src.config.db_connection import load_data_db
from pyspark.ml.classification import RandomForestClassifier, RandomForestClassificationModel
from pyspark.ml.pipeline import Pipeline, PipelineModel
from pyspark.ml.feature import StringIndexer, IndexToString
from pyspark.sql.functions import lit, col, count, when, isnan, current_timestamp, unix_timestamp, min, max, minute, second, concat
from pyspark.sql.types import StructField, StructType, StringType, IntegerType

24/01/02 12:43:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
shutil.copytree('./user_exp_artifact/model_trainer/UE_model', './model/best_model/', dirs_exist_ok=True)

'./model/best_model/'

In [29]:
#model = PipelineModel.load('./user_exp_artifact/model_trainer/UE_model*')
model = PipelineModel.load('./model/best_model/UE_model*')

In [9]:
transformed_pipeline = PipelineModel.load('./user_exp_artifact/data_transformation/object/data_transform_pipeline.pkl*')

In [33]:
#new_stages = transformed_pipeline.stages + model.stages
#new_stages
#transformed_pipeline.stages
out_trans = model.stages[-2]
out_trans.labels

['UBE', 'UGE']

In [15]:
final_model = Pipeline(stages=new_stages)

In [5]:
transformed_pipeline.stages = new_stages

In [5]:
data = spark_session.read.csv('../user_exp_artifact/data_validation/User_final_data.csv*', header=True, inferSchema=True)
test = data.filter('indicator1 < 100')
data.count()

126632

In [6]:
#data1 = data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns]).collect()
#data1[0][0]
data = data.withColumn('specifictime1', concat(minute('specifictime'), lit(':'), second('specifictime')))
data.show()

+-------------------+----------+----------+----------+----------+----------+----------+----------+----------+---------+------+-----------------+------------------+------------------+------------------+-----------------+------------------+--------------------+--------------------+-------------+
|       specifictime|indicator1|indicator2|indicator3|indicator4|indicator5|indicator6|indicator7|indicator8|     user|result|   indicator1_avg|    indicator2_avg|    indicator3_avg|    indicator4_avg|   indicator5_avg|    indicator6_avg|      indicator7_avg|      indicator8_avg|specifictime1|
+-------------------+----------+----------+----------+----------+----------+----------+----------+----------+---------+------+-----------------+------------------+------------------+------------------+-----------------+------------------+--------------------+--------------------+-------------+
|2021-06-10 18:04:59|      17.0|       2.0|       5.0|      18.0|       2.0|      17.0|       0.0|       0.0|user3_

In [8]:
#rows = data1[0]
#rw1 = rows.asDict()
#data.withColumn('submit_date', current_timestamp()).select('submit_date').show(truncate=False)
#rl = tuple(rows.asDict().values())
#rl
#data.select('specifictime').sort('specifictime').show()
#data.select(min('specifictime'), max('specifictime')).show()
data.withColumn('minute', minute('specifictime')).show()

+-------------------+----------+----------+----------+----------+----------+----------+----------+----------+---------+------+-----------------+------------------+------------------+------------------+-----------------+------------------+--------------------+--------------------+-------------+------+
|       specifictime|indicator1|indicator2|indicator3|indicator4|indicator5|indicator6|indicator7|indicator8|     user|result|   indicator1_avg|    indicator2_avg|    indicator3_avg|    indicator4_avg|   indicator5_avg|    indicator6_avg|      indicator7_avg|      indicator8_avg|specifictime1|minute|
+-------------------+----------+----------+----------+----------+----------+----------+----------+----------+---------+------+-----------------+------------------+------------------+------------------+-----------------+------------------+--------------------+--------------------+-------------+------+
|2021-06-10 18:04:59|      17.0|       2.0|       5.0|      18.0|       2.0|      17.0|       

In [48]:
rw2 = [key for key, val in rw1.items() if val == 0]
rw2

['specifictime',
 'indicator1',
 'indicator2',
 'indicator3',
 'indicator4',
 'indicator5',
 'indicator6',
 'indicator7',
 'indicator8',
 'user',
 'result',
 'indicator1_avg',
 'indicator2_avg',
 'indicator3_avg',
 'indicator4_avg',
 'indicator5_avg',
 'indicator6_avg',
 'indicator7_avg',
 'indicator8_avg']

In [22]:
pred = model.transform(test)
pred.show()

+-------------------+----------+----------+----------+----------+----------+----------+----------+----------+---------+------+-----------------+------------------+------------------+------------------+-----------------+-----------------+-------------------+-------------------+---------------+--------------------+--------------------+-----+-------------+-----------+----------+
|       specifictime|indicator1|indicator2|indicator3|indicator4|indicator5|indicator6|indicator7|indicator8|     user|result|   indicator1_avg|    indicator2_avg|    indicator3_avg|    indicator4_avg|   indicator5_avg|   indicator6_avg|     indicator7_avg|     indicator8_avg|en_specifictime|        num_features|             feature|label|rawPrediction|probability|prediction|
+-------------------+----------+----------+----------+----------+----------+----------+----------+----------+---------+------+-----------------+------------------+------------------+------------------+-----------------+-----------------+-----

In [33]:
shutil.copytree('./user_exp_artifact/model_trainer/', './test', dirs_exist_ok=True)

'./test'

In [3]:
pred_df = spark_session.read.parquet('./output/pred.parquet')
pred_df.show()


23/11/22 07:53:33 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------------------+----------+----------+----------+----------+----------+----------+----------+----------+-------+-----------------+-----------------+-----------------+----------------+-----------------+------------------+--------------------+--------------------+------+---------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
|       specifictime|indicator1|indicator2|indicator3|indicator4|indicator5|indicator6|indicator7|indicator8|   user|   indicator1_avg|   indicator2_avg|   indicator3_avg|  indicator4_avg|   indicator5_avg|    indicator6_avg|      indicator7_avg|      indicator8_avg|result|en_specifictime|        num_features|             feature|label|       rawPrediction|         probability|prediction|
+-------------------+----------+----------+----------+----------+----------+----------+----------+----------+-------+-----------------+-----------------+-----------------+----------------+-----------------+----------

In [6]:
pred = pred_df.select('prediction')
pred1 = pred.groupBy('prediction').count().sort(col('count').desc())

In [33]:
adf = spark_session.createDataFrame(data=[("a", 1), ("b", 2), ("d", 3), ("", 4), ("e", 5), ("e", 5)], schema=StructType([StructField("data", StringType(), True), StructField("num", IntegerType(), True)]))
adf.show()

+----+---+
|data|num|
+----+---+
|   a|  1|
|   b|  2|
|   d|  3|
|    |  4|
|   e|  5|
|   e|  5|
+----+---+



In [37]:
adf.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in adf.columns]).show()

+----+---+
|data|num|
+----+---+
|   0|  0|
+----+---+



In [3]:
string_idx = StringIndexer(inputCol="data", outputCol="en_data")
adf_1 = string_idx.fit(adf).transform(adf)
adf_1 = adf_1.withColumn("en_data", col("en_data").cast("int"))

In [4]:
adf = adf.dropDuplicates()
adf.show()

+----+---+
|data|num|
+----+---+
|   a|  1|
|   b|  2|
|   d|  3|
|   c|  4|
|   e|  5|
+----+---+



In [11]:
adf_2 = adf_1.select(*["en_data", "data"]).collect()
adf_2

[Row(en_data=1, data='a'),
 Row(en_data=2, data='b'),
 Row(en_data=4, data='d'),
 Row(en_data=3, data='c'),
 Row(en_data=0, data='e'),
 Row(en_data=0, data='e')]

In [10]:
adfs = adf_2[0]
adfs[0]

'a'

In [10]:
adf.first()[0]

'a'

In [43]:
pair = {}
for dat in adf_2:
    pair[dat[1]] = dat[0]

In [44]:
pair

{0: 'a', 1: 'b', 3: 'd', 2: 'c', 4: 'e'}

In [5]:
incident_df = spark_session.read.csv('./Incident.xls', header=True, inferSchema=True)

In [15]:
incident_count_df = incident_df.groupBy('INCIDENT_NUMBER').count()
incident_count_df = incident_count_df.withColumn('count1', col('count')).drop('count')

In [36]:
incident_count_df1 = incident_count_df.groupBy('count1').count()
incident_count_df1 = incident_count_df1.withColumn('No of Transactions per Incident', col('count1').cast('int')).withColumn('No of Incidents', col('count').cast('int')).drop(*['count1', 'count'])
incident_count_df1.repartition(1).write.csv('./Incident_count.csv', header=True)

In [24]:
stime = time.time()
stime

1700620549.6098254

In [25]:
etime = time.time()
etime

1700620553.4298513

In [26]:
etime - stime

3.82002592086792

In [4]:
help(adf.rdd.map)

Help on method map in module pyspark.rdd:

map(f, preservesPartitioning=False) method of pyspark.rdd.RDD instance
    Return a new RDD by applying a function to each element of this RDD.
    
    Examples
    --------
    >>> rdd = sc.parallelize(["b", "a", "c"])
    >>> sorted(rdd.map(lambda x: (x, 1)).collect())
    [('a', 1), ('b', 1), ('c', 1)]



In [4]:
test_p = spark_session.read.parquet('../user_exp_artifact/data_transformation/data/test_user_df.parquet*')
test_p.show()

+-------------------+----------+----------+----------+----------+----------+----------+----------+----------+---------+------+-----------------+------------------+------------------+------------------+-----------------+-----------------+-------------------+-------------------+-----+
|       specifictime|indicator1|indicator2|indicator3|indicator4|indicator5|indicator6|indicator7|indicator8|     user|result|   indicator1_avg|    indicator2_avg|    indicator3_avg|    indicator4_avg|   indicator5_avg|   indicator6_avg|     indicator7_avg|     indicator8_avg|label|
+-------------------+----------+----------+----------+----------+----------+----------+----------+----------+---------+------+-----------------+------------------+------------------+------------------+-----------------+-----------------+-------------------+-------------------+-----+
|2021-06-10 18:04:59|      11.0|       6.0|       8.0|       3.0|      26.0|      11.0|       0.0|       0.0|user4_UBE|   UBE|22.24534555563388|178.

In [2]:
df = spark_session.read.parquet('../user_exp_artifact/data_transformation/data/train_user_df.parquet*')

In [3]:
df.show()

+--------------------+-----+
|             feature|label|
+--------------------+-----+
|[0.52384490493926...|  0.0|
|[1.10589479931622...|  0.0|
|[1.39691974650470...|  0.0|
|[1.39691974650470...|  0.0|
|[1.39691974650470...|  0.0|
|[1.39691974650470...|  0.0|
|[1.39691974650470...|  0.0|
|[1.39691974650470...|  0.0|
|[1.39691974650470...|  0.0|
|[1.39691974650470...|  0.0|
|[1.39691974650470...|  0.0|
|[1.39691974650470...|  0.0|
|[1.39691974650470...|  0.0|
|[1.39691974650470...|  0.0|
|[1.68794469369318...|  0.0|
|[1.68794469369318...|  0.0|
|[1.68794469369318...|  0.0|
|[1.68794469369318...|  0.0|
|[1.68794469369318...|  0.0|
|[1.68794469369318...|  0.0|
+--------------------+-----+
only showing top 20 rows



In [6]:
df1 = df.select(['label', 'feature'])
df1.show()

+-----+--------------------+
|label|             feature|
+-----+--------------------+
|  0.0|[0.52384490493926...|
|  0.0|[1.10589479931622...|
|  0.0|[1.39691974650470...|
|  0.0|[1.39691974650470...|
|  0.0|[1.39691974650470...|
|  0.0|[1.39691974650470...|
|  0.0|[1.39691974650470...|
|  0.0|[1.39691974650470...|
|  0.0|[1.39691974650470...|
|  0.0|[1.39691974650470...|
|  0.0|[1.39691974650470...|
|  0.0|[1.39691974650470...|
|  0.0|[1.39691974650470...|
|  0.0|[1.39691974650470...|
|  0.0|[1.68794469369318...|
|  0.0|[1.68794469369318...|
|  0.0|[1.68794469369318...|
|  0.0|[1.68794469369318...|
|  0.0|[1.68794469369318...|
|  0.0|[1.68794469369318...|
+-----+--------------------+
only showing top 20 rows



In [7]:
if 2>3 and 3>4:
    True